In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('reviews.csv')

In [5]:
df.head()

,titletext,label
0,pinnacle seems cushioning husband likes better...,0
1,great price product though sizes tend bigger b...,0
2,good fit even washing hot water force shrinkin...,0
3,great shoe ive nikes always pleased comfort pe...,0
4,looks bit nicer picture light way held box won...,0


In [6]:
df['label'].value_counts()

1    3581
0    3580
Name: label, dtype: int64

In [7]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 4.2 MB/s 
     |████████████████████████████████| 511.7 MB 5.3 kB/s 
     |████████████████████████████████| 1.6 MB 53.2 MB/s 
     |████████████████████████████████| 438 kB 69.2 MB/s 
     |████████████████████████████████| 5.8 MB 37.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['titletext'],df['label'], stratify=df['label'], test_size=0.2)

In [10]:
y_train.value_counts()

1    2864
0    2864
Name: label, dtype: int64

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [13]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [15]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [16]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
162/162 [==============================] - 51s 240ms/step - loss: 0.6247 - accuracy: 0.6407 - precision: 0.6355 - recall: 0.6533 - val_loss: 0.5526 - val_accuracy: 0.7452 - val_precision: 0.7352 - val_recall: 0.7946
Epoch 2/10
162/162 [==============================] - 38s 234ms/step - loss: 0.5269 - accuracy: 0.7437 - precision: 0.7318 - recall: 0.7663 - val_loss: 0.5211 - val_accuracy: 0.7155 - val_precision: 0.7557 - val_recall: 0.6667
Epoch 3/10
162/162 [==============================] - 38s 234ms/step - loss: 0.4984 - accuracy: 0.7546 - precision: 0.7400 - recall: 0.7818 - val_loss: 0.4949 - val_accuracy: 0.7644 - val_precision: 0.7411 - val_recall: 0.8384
Epoch 4/10
162/162 [==============================] - 38s 234ms/step - loss: 0.4817 - accuracy: 0.7668 - precision: 0.7527 - recall: 0.7920 - val_loss: 0.4912 - val_accuracy: 0.7487 - val_precision: 0.7703 - val_recall: 0.7340
Epoch 5/10
162/162 [==============================] - 38s 234ms/step - loss: 0.4764 - accura

In [17]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

45/45 [==============================] - 10s 205ms/step


array([0, 1, 0, ..., 1, 0, 1])

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       0.73      0.85      0.79       716
           1       0.82      0.68      0.75       717

    accuracy                           0.77      1433
   macro avg       0.78      0.77      0.77      1433
weighted avg       0.78      0.77      0.77      1433



In [19]:
model.save_weights('bert_trained.h5')

In [20]:
sample_dataset = [
 'I like this appliance and b'
]
test_check = model.predict(sample_dataset)
print(np.where(test_check > 0.5, 'generated_review', 'real_review'))

1/1 [==============================] - 1s 791ms/step
[['generated_review']]
